<a href="https://colab.research.google.com/github/danisshai/fastai-tests/blob/main/notebooks/Mnist_all_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! [ -e /content ] && pip install -Uqq fastbook


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#hide
from fastai.vision.all import *
from fastbook import *


In [10]:
path = untar_data(URLs.MNIST)

In [11]:
def get_imgs_tensor(file_path):
  return torch.stack([ tensor(Image.open(img_path)) for img_path in file_path.ls()]).float()/255

In [12]:
# 0-9
  # 0 = [1,0,0,0,0,0,0,0,0,0]
  # 1 = [0,1,0,0,0,0,0,0,0,0]
  # 2 = [0,0,1,0,0,0,0,0,0,0]
  # ...
  # first position is the stack and second the encodding

train_set = list(zip([get_imgs_tensor(path) for path in (path/"training").ls().sorted()],
                 nn.functional.one_hot(torch.arange(0,10))))
valid_set = list(zip([get_imgs_tensor(path) for path in (path/"testing").ls().sorted()],
                 nn.functional.one_hot(torch.arange(0,10))))

In [13]:
train_set[0][0].shape

torch.Size([5923, 28, 28])

In [14]:
x_train = torch.cat([cat[0] for cat in train_set]).view(-1, 28*28)
y_train = torch.cat([number_set[1].repeat(len(number_set[0]),1) for number_set in train_set])

In [15]:
x_train.shape, y_train.shape

(torch.Size([60000, 784]), torch.Size([60000, 10]))

In [16]:
x_valid = torch.cat([cat[0] for cat in valid_set]).view(-1, 28*28)
y_valid = torch.cat([number_set[1].repeat(len(number_set[0]),1) for number_set in valid_set])
x_valid.shape, y_valid.shape

(torch.Size([10000, 784]), torch.Size([10000, 10]))

In [17]:
dl = DataLoader(list(zip(x_train,y_train)), batch_size=256)
dl_valid = DataLoader(list(zip(x_valid,y_valid)),  batch_size=256)

In [18]:
dls = DataLoaders(dl, dl_valid)

In [19]:
def mnist_loss(predictions, targets):
     predictions = predictions.sigmoid()
     return torch.where(targets==1, 1-predictions, predictions).mean()
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()

    max_index_xb = torch.argmax(xb, dim=-1, keepdim=True)
    max_index_yb = torch.argmax(yb, dim=-1, keepdim=True)
    correct = max_index_xb == max_index_yb
    return correct.float().mean()

In [20]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30*30),
    nn.ReLU(),
    nn.Linear(30*30,20*20),
    nn.ReLU(),
    nn.Linear(20*20,10*10),
    nn.ReLU(),
    nn.Linear(10*10,10)
)

simple_net2 = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,10)
)

In [21]:
learn = Learner(dls,  simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [22]:
lr=0.1
learn.fit(100, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.135711,0.103082,0.107700,00:08
1,0.100885,0.100654,0.097500,00:05
2,0.100261,0.100336,0.113500,00:07


KeyboardInterrupt: ignored

In [ ]:
learn.parameters